## Contribution analysis in the foreground system

### Massimo Pizzol, 2024

In [1]:
# Note: must be environment bw2
import brightway2 as bw
import pandas as pd
import numpy as np

Setting up the project with databases, a LCIA method, and some activities

In [2]:
bw.projects.set_current("advlca23") # my project with ei3.9 conseq

In [45]:
# a method for LCIA
mymethod = ('ReCiPe 2016 v1.03, midpoint (H)', 'climate change', 'global warming potential (GWP1000)')

In [34]:
# Make a list of activities from ecoinvent, chosen randomly
myacts = [bw.Database('ecoinvent 3.9 conseq').random() for _ in range(5)]
myacts

['treatment of wastewater from hard fibreboard production, wastewater treatment' (cubic meter, RER, None),
 'market for 6-benzyladenine' (kilogram, GLO, None),
 'market for electricity, medium voltage' (kilowatt hour, AT, None),
 'market for cobalt carbonate' (kilogram, GLO, None),
 'aluminium chloride production' (kilogram, GLO, None)]

### One static score

In [46]:
# Forthe first activity in the list
myact = bw.Database(myacts[1]['database']).get(myacts[1]['code'])
print(myact)
functional_unit = {myact: 1} 
lca = bw.LCA(functional_unit, mymethod)
lca.lci()
lca.lcia()
print(lca.score)

'market for 6-benzyladenine' (kilogram, GLO, None)
8.716756507694592


Function to get LCA results

In [23]:
def dolcacalc(myact, mydemand, mymethod):
    my_fu = {myact: mydemand} 
    lca = bw.LCA(my_fu, mymethod)
    lca.lci()
    lca.lcia()
    return lca.score

# For WtT
def getLCAresults(list_acts, mymethod):
    
    all_activities = []
    results = []
    for a in list_acts:
        act = bw.Database(a[0]).get(a[1])
        all_activities.append(act['name'])
        results.append(dolcacalc(act,1,mymethod)) # 1 stays for one unit of each process
        #print(act['name'])
     
    results_dict = dict(zip(all_activities, results))
    
    return results_dict

# Contribution Analysis

One might want to know how much each exchange of an activity contributes to the total impact of such "parent" activity.

What I do here is iterating through the exchanges of the "parent" activity and calculate impact scores for each exchange as if this was the functional unit, using the amount used in input to the "parent" activity

The tings to notice is that a different approach is used for biosphere and technosphere exchanges

In [36]:
ca_dict = {}

for act in myacts:
    
    exc_list = []
    contr_list = []

    for exc in list(act.exchanges()):
        
        if exc['type'] == 'biosphere':
            
            col = lca.activity_dict[exc['output']] # find column index of A matrix for the activity
            row = lca.biosphere_dict[exc['input']] # find row index of B matrix for the exchange
            contr_score = lca.biosphere_matrix[row,col] * lca.characterization_matrix[row,row]
            contr_list.append((exc['input'], exc['type'], exc['amount'], contr_score))
            
        elif exc['type'] == 'substitution':
            
            contr_score = dolcacalc(bw.Database(exc['input'][0]).get(exc['input'][1]), exc['amount'], mymethod)
            contr_list.append((exc['input'], exc['type'], exc['amount'], -contr_score))
            
        else:
            
            contr_score = dolcacalc(bw.Database(exc['input'][0]).get(exc['input'][1]), exc['amount'], mymethod)
            contr_list.append((exc['input'], exc['type'], exc['amount'], contr_score))
        
    ca_dict[act['code']] =  contr_list

### We can check one activity

Using some nice formatting

In [42]:
df = pd.DataFrame(ca_dict[myacts[1]['code']], columns = ['input','type','amount','contribution'])
df

,input,type,amount,contribution
0,"(ecoinvent 3.9 conseq, db9a71a0edf13c89a31140e...",production,1.0000,8.716757
1,"(ecoinvent 3.9 conseq, db232fc4d8484ff70f90aaf...",technosphere,1.0000,8.659267
2,"(ecoinvent 3.9 conseq, f005944b534083aff9e9922...",technosphere,0.3091,0.017582
3,"(ecoinvent 3.9 conseq, 9ef3e3988eccdfcbf356d2d...",technosphere,0.0246,0.001317
4,"(ecoinvent 3.9 conseq, 70f7992d941cafc4dd70f28...",technosphere,0.2088,0.032328
5,"(ecoinvent 3.9 conseq, 390ba2d0a2107982360b357...",technosphere,0.5990,0.006263


Let's check it gives the right result, and the sum of all the exchanges equals the impact of the production activity

It's not perfectly close. I guess due to rounding errors? The difference is very very small.

In [43]:
print(df.loc[df['type'] == 'production']['contribution'].sum())
print(df.loc[df['type'] != 'production']['contribution'].sum())

8.716756507694592
8.716756507406608


Here below I calculate the contribution in percentage. 

In [44]:
df['%_contribution'] = 100 * df['contribution'] / df.loc[df['type'] == 'production']['contribution'].sum()
df

,input,type,amount,contribution,%_contribution
0,"(ecoinvent 3.9 conseq, db9a71a0edf13c89a31140e...",production,1.0000,8.716757,100.000000
1,"(ecoinvent 3.9 conseq, db232fc4d8484ff70f90aaf...",technosphere,1.0000,8.659267,99.340477
2,"(ecoinvent 3.9 conseq, f005944b534083aff9e9922...",technosphere,0.3091,0.017582,0.201708
3,"(ecoinvent 3.9 conseq, 9ef3e3988eccdfcbf356d2d...",technosphere,0.0246,0.001317,0.015104
4,"(ecoinvent 3.9 conseq, 70f7992d941cafc4dd70f28...",technosphere,0.2088,0.032328,0.370867
5,"(ecoinvent 3.9 conseq, 390ba2d0a2107982360b357...",technosphere,0.5990,0.006263,0.071844


In [341]:
mymethod

('EF v3.1', 'climate change', 'global warming potential (GWP100)')

In [342]:
bw.Database('biosphere3').get('cc6a1abb-b123-4ca6-8f16-38209df609be')

'Carbon dioxide, in air' (kilogram, None, ('natural resource', 'in air'))

In [343]:
col = lca.activity_dict[('Fuels_db_WtT_MJ', 'eMeOH_DAC')] # find column index of A matrix for the activity
row = lca.biosphere_dict[('biosphere3', 'cc6a1abb-b123-4ca6-8f16-38209df609be')] # find row index of B matrix for the exchange
print('row:', row, 'column:', col)
print('value of the exchange:', lca.biosphere_matrix[row,col])
print('Characterisation factor', lca.characterization_matrix[row,row])

row: 83 column: 18860
value of the exchange: -0.11299999803304672
Characterisation factor 0.0


In [344]:
#here you see which exchanges have cfs.

print(lca.characterization_matrix)
#('biosphere3', 'af01e564-f816-4906-bd4f-b7c932f926b9')

  (70, 70)	0.006000000052154064
  (82, 82)	1.0
  (96, 96)	20.600000381469727
  (163, 163)	0.43700000643730164
  (164, 164)	1530.0
  (165, 165)	164.0
  (166, 166)	1.2999999523162842
  (167, 167)	12400.0
  (171, 171)	6.340000152587891
  (274, 274)	2.430000066757202
  (275, 275)	7200.0
  (276, 276)	1960.0
  (277, 277)	11.199999809265137
  (278, 278)	12500.0
  (279, 279)	160.0
  (280, 280)	29.799999237060547
  (281, 281)	27.0
  (282, 282)	2200.0
  (283, 283)	7380.0
  (284, 284)	6230.0
  (285, 285)	14600.0
  (302, 302)	0.48100000619888306
  (312, 312)	17400.0
  (401, 401)	0.019999999552965164
  (460, 460)	25200.0
  :	:
  (831, 831)	29.799999237060547
  (832, 832)	5.539999961853027
  (833, 833)	5.539999961853027
  (834, 834)	2200.0
  (835, 835)	7380.0
  (836, 836)	14600.0
  (846, 846)	0.48100000619888306
  (895, 895)	0.019999999552965164
  (896, 896)	0.019999999552965164
  (947, 947)	25200.0
  (948, 948)	25200.0
  (1706, 1706)	29.799999237060547
  (1707, 1707)	1.0
  (1708, 1708)	-1.0
  (1710

In [282]:
for i in bw.Database('biosphere3').search('carbon dioxide fossil'):
    print(i)
    print(i['code'])

'Carbon dioxide, fossil' (kilogram, None, ('air', 'low population density, long-term'))
e259263c-d1f1-449f-bb9b-73c6d0a32a00
'Carbon dioxide, fossil' (kilogram, None, ('air', 'non-urban air or from high stacks'))
aa7cac3a-3625-41d4-bc54-33e2cf11ec46
'Carbon dioxide, fossil' (kilogram, None, ('air',))
349b29d1-3e58-4c66-98b9-9d1a076efd2e
'Carbon dioxide, fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere'))
16eeda8a-1ea2-408e-ab37-2648495058dd
'Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground'))
f9749677-9c9f-4678-ab55-c607dfdc2cb9
'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'non-urban air or from high stacks'))
d6235194-e4e6-4548-bfa3-ac095131aef4
'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'low population density, long-term'))
28e1e2d6-97ad-4dfd-932a-9edad36dcab9
'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere'))
4e1f0bb0-2703-4303-bf86-972d810612cf
'Carbon dioxide, non-f

### Monte Carlo simulations

In [6]:
# Uncertainty type: an interger that tells you what type of uncertainty (see website for list). 2 = lognormal distribution https://docs.brightway.dev/en/latest/content/theory/uncertainty.html
# loc: median
# scale: geometric standard deviation
# negative: is the value negative (TRUE) or not (FALSE)

# https://github.com/PoutineAndRosti/Brightway-Seminar-2017/blob/master/Day%201%20PM/Brightway%20and%20uncertainty.ipynb
# https://stats-arrays.readthedocs.io/en/latest/index.html

# What I did: 
# Calculated the natural log of the median and natural log of the geometric standard deviation and used these for 'loc' and 'scale' values

In [19]:
mymethod = EF31[1]

In [20]:
meoh = Fuels_db_TtW_MJ.get('MeOH_combustion')
meoh_exc = list(meoh.exchanges())[0]
meoh_exc

Exchange: 3.06e-06 kilogram 'Particulate Matter, < 2.5 um' (kilogram, None, ('air', 'non-urban air or from high stacks')) to 'MeOH_combustion' (megajoule, None, None)>

In [21]:
meoh_exc.uncertainty

{'uncertainty type': 2, 'loc': -12.3, 'scale': 0.33, 'negative': False}

In [22]:
mc = bw.MonteCarloLCA({meoh: 1}, mymethod)  # Monte Carlo class
#mc_results = [next(mc) for x in range(10)] 
next(mc)

InvalidParamsError: Real, positive scale (sigma) values are required for lognormal uncertainties.

In [ ]:
fus = [] # list of functional units
for a in acts:
    act = Database('CCU').get(a)
    functional_unit = {act: 1} # one unit of each process
    fus.append(functional_unit)

The other way of doing it (multiLCA)

In [48]:
eMeOH_DOC_WtT_MJ = Fuels_db_WtT_MJ.get('eMeOH_DOC')
list(eMeOH_DOC_WtT_MJ.exchanges())

[Exchange: 0.0158 kilogram 'eH2' (kilogram, None, None) to 'eMeOH_DOC' (megajoule, None, None)>,
 Exchange: 0.075 kilowatt hour 'market for electricity, high voltage' (kilowatt hour, DK, None) to 'eMeOH_DOC' (megajoule, None, None)>,
 Exchange: 0.00442 kilogram 'market for platinum' (kilogram, GLO, None) to 'eMeOH_DOC' (megajoule, None, None)>,
 Exchange: -0.113 kilogram 'Carbon dioxide, in air' (kilogram, None, ('natural resource', 'in air')) to 'eMeOH_DOC' (megajoule, None, None)>,
 Exchange: 0.000907 unit 'DAC' (unit, None, None) to 'eMeOH_DOC' (megajoule, None, None)>,
 Exchange: 0.0471 kilowatt hour 'market for electricity, high voltage' (kilowatt hour, DK, None) to 'eMeOH_DOC' (megajoule, None, None)>,
 Exchange: 3.08e-12 unit 'methanol factory construction' (unit, GLO, None) to 'eMeOH_DOC' (megajoule, None, None)>,
 Exchange: 0.1 ton kilometer 'transport, pipeline, onshore, petroleum' (ton kilometer, RER, None) to 'eMeOH_DOC' (megajoule, None, None)>,
 Exchange: 1.0 megajoule 'e

In [49]:
#list_functional_units = [{eMeOH_DOC_WtT_MJ.key:1}]
#list_methods = CML2016
#bw.calculation_setups['multiimpact'] = {'inv':list_functional_units, 'ia':list_methods}

In [50]:
# bw.calculation_setups['multiimpact']

In [51]:
# myMultiLCA = bw.MultiLCA('multiimpact')

In [52]:
# myMultiLCA.results

In [53]:
# my_output = pd.DataFrame(index=CML2016, columns=[VLSFO_WtT_MJ['name']], data=myMultiLCA.results.T)

In [54]:
# my_output.to_excel('VLSFO_results_CML2016.xlsx')